# Snowdamage Detection using Satellite Imagery

This Jupyter notebook demonstrates how to detect snowdamage areas in satellite imagery using a combination of Python libraries and tools. The primary purpose of this notebook is to process and visualize the results of snowdamage detection algorithms, as well as analyze the performance of the detection methods using various metrics.

## Workflow

The notebook is organized into the following sections:

    - Import necessary libraries and tools.
    - Read and preprocess the input satellite imagery.
    - Apply clearcut detection algorithms on the input data.
    - Visualize the detected clearcut areas.
    - Compute and analyze various performance metrics for the detection algorithms.
    - Compare the results of different detection methods and visualize them in a single plot.

Throughout the notebook, various helper functions and custom tools are used to simplify the code and make the workflow more efficient. The notebook also includes several visualizations, such as maps and Receiver Operating Characteristic (ROC) curves, to help users understand the results and performance of the clearcut detection methods.

To start using the notebook, simply run each code cell in order, following the comments and instructions provided.



## Importing Libraries and Tools

In [ ]:
import os
import re
import sys; sys.argv=['']

project_root = '/local_home/kuzu_ri/GIT_REPO/representlib'
sys.path.append(project_root)

from represent.tools.utils_uc1 import process_and_plot, read_image,interpolate_dataframe


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.patches import Patch
from matplotlib.cm import ScalarMappable
import matplotlib.font_manager as fm
import seaborn as sns
sns.set_style("whitegrid")



from skimage import morphology
from skimage.morphology import dilation, disk
from sklearn.metrics import confusion_matrix
from scipy import ndimage

import pandas as pd
import rasterio

from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from rasterio import warp
from pyproj import Transformer




## Image Processing and Visualization Functions

In [ ]:

def normalize_band(array, lower_percentile=2, upper_percentile=98):
    """
    Normalize an input band array based on the given percentile values.
    
    Args:
        array (numpy array): The input band array.
        lower_percentile (int, optional): The lower percentile value. Defaults to 2.
        upper_percentile (int, optional): The upper percentile value. Defaults to 98.
        
    Returns:
        numpy array: The normalized band array.
    """
    lower, upper = np.percentile(array, (lower_percentile, upper_percentile))
    return np.clip((array - lower) / (upper - lower), 0, 1)

def plot_rgb_image(image_path):
    
    """
    Plot an RGB image from a given Sentinel-2 Image path.
    
    Args:
        image_path (str): The path to the input image file.
        
    Returns:
        numpy array: The plotted RGB image array.
    """
    
    # Open the Sentinel-1 data file
    with rasterio.open(image_path) as src:
        VV = src.read(1)
        VH = src.read(2)
        
        dop = (VV / (VV + VH + np.finfo(float).eps))
        rvi = (np.sqrt(dop)) * ((4 * (VH)) / (VV + VH +np.finfo(float).eps))
        
        blue = rvi
        green = VV
        red = VH
        
        extent = src.bounds
        height, width = blue.shape
    
    # Enlarge the extent of the plot by 100% on each dimension
    x_min, y_min, x_max, y_max = extent
    x_range = x_max - x_min
    y_range = y_max - y_min
    extent = (x_min - x_range, x_max + x_range, y_min - y_range, y_max + y_range)

    red = normalize_band(red)
    green = normalize_band(green)
    blue = normalize_band(blue)

    rgb = np.dstack((red, green, blue))
    
    
    # Calculate the aspect ratio
    aspect_ratio = float(width) / float(height)

    # Calculate the figure size based on the aspect ratio
    base_fig_size = 14
    fig_size = (base_fig_size * aspect_ratio, base_fig_size)

    # Create the plot with the RGB image
    fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': ccrs.epsg(3067)})
    ax.imshow(rgb, extent=extent, transform=ccrs.epsg(3067), origin='upper')

    # Set the extent of the plot to include the entire forest mask image
    ax.set_extent(extent, crs=ccrs.epsg(3067))

    # Add gridlines and lat/long labels
    gridlines = ax.gridlines(draw_labels=True)
    gridlines.xformatter = LongitudeFormatter()
    gridlines.yformatter = LatitudeFormatter()

    # Define the scale bar properties
    scalebar = ScaleBar(1, 'm', length_fraction=0.25, height_fraction=0.005, location='lower right')


    # Add the scale bar to the plot
    ax.add_artist(scalebar)

    plt.tight_layout()
    
    return rgb

    
def plot_zoomed_input(input_image,north, west, east, south,loc='upper right'):
    """
    Plot a zoomed input image based on the given geographic coordinates.
    
    Args:
        input_image (numpy array): The input image array.
        north (float): The northern boundary.
        west (float): The western boundary.
        east (float): The eastern boundary.
        south (float): The southern boundary.
        loc (str, optional): The location of the legend. Defaults to 'upper right'.
    """
        
    # Convert the N, W, E, and S values to x and y in the EPSG:3067 coordinate system
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3067", always_xy=True)
    x_min_zoom, y_min_zoom = transformer.transform(west, south)
    x_max_zoom, y_max_zoom = transformer.transform(east, north)
    boundary_cmap = ListedColormap([(0, 0, 0, 0), (0, 0, 0, 1)])

    fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': ccrs.epsg(3067)})
    ax.imshow(input_image, extent=extent, transform=ccrs.epsg(3067), origin='upper')
    
    ax.set_extent([x_min_zoom, x_max_zoom, y_min_zoom, y_max_zoom], crs=ccrs.epsg(3067))

    # Add gridlines and lat/long labels
    gridlines = ax.gridlines(draw_labels=True)
    gridlines.xformatter = LongitudeFormatter()
    gridlines.yformatter = LatitudeFormatter()

    # Add scalebar
    scalebar = ScaleBar(1, 'm', length_fraction=0.25, height_fraction=0.005, location='lower right')
    ax.add_artist(scalebar)
 
    fig.canvas.draw()

    #plt.show()
    

def plot_zoomed_result(north, west, east, south,loc='upper right'):
    """
    Plot a zoomed result image based on the given geographic coordinates.
    
    Args:
        north (float): The northern boundary.
        west (float): The western boundary.
        east (float): The eastern boundary.
        south (float): The southern boundary.
        loc (str, optional): The location of the legend. Defaults to 'upper right'.
    """
    
    # Convert the N, W, E, and S values to x and y in the EPSG:3067 coordinate system
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3067", always_xy=True)
    x_min_zoom, y_min_zoom = transformer.transform(west, south)
    x_max_zoom, y_max_zoom = transformer.transform(east, north)
    boundary_cmap = ListedColormap([(0, 0, 0, 0), (0, 0, 0, 1)])

    #custom_cmap = LinearSegmentedColormap.from_list("custom_colormap", [custom_colormap(i) for i in range(-1, 131)], N=131)
    #custom_cmap = plt.cm.get_cmap("hot_r", 130)

    fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': ccrs.epsg(3067)})
    ax.imshow(rgb_image, extent=extent, transform=ccrs.epsg(3067), origin='upper')
    #ax.imshow(boundaries, cmap=boundary_cmap, alpha=0.7, extent=extent, transform=ccrs.epsg(3067), origin='upper')
    
    ax.set_extent([x_min_zoom, x_max_zoom, y_min_zoom, y_max_zoom], crs=ccrs.epsg(3067))

    # Add gridlines and lat/long labels
    gridlines = ax.gridlines(draw_labels=True)
    gridlines.xformatter = LongitudeFormatter()
    gridlines.yformatter = LatitudeFormatter()

    # Add scalebar
    scalebar = ScaleBar(1, 'm', length_fraction=0.25, height_fraction=0.005, location='lower right')
    ax.add_artist(scalebar)


    # Create a custom legend item for the boundaries
    # Create a legend with colors and labels
    legend_elements = [Patch( facecolor='green',edgecolor='gray', label='TP'),
                   Patch( facecolor='cyan',edgecolor='gray', label='TN'),
                   Patch( facecolor='orange',edgecolor='gray', label='FN'),
                   Patch( facecolor='magenta',edgecolor='gray', label='FP'),
                   #plt.Rectangle((0,0),1,1, color='black', label='Non-forest Mask'),
                  ]


    # Add the legend to the plot
    ax.legend(handles=legend_elements, loc=loc)
    fig.canvas.draw()

    #plt.show()

## Setting Image Paths and Loading Images

In [ ]:
pre_s1_path = project_root+"/represent/data/UC1/snow_damage/S1/S1A_IW_GRDH_1SDV_20171124_epsg3067.tif"
post_s1_path = project_root+"/represent/data/UC1/snow_damage/S1/S1A_IW_GRDH_1SDV_20180312_epsg3067.tif"

ground_truth_disturbance_path=project_root+"/represent/data/UC1/snow_damage/GT/disturbed_area_sample.tif"
ground_truth_intact_path=project_root+"/represent/data/UC1/snow_damage/GT/intact_area_sample.tif"
forest_mask_path=project_root+"/represent/data/UC1/snow_damage/GT/forest_mask.tif"
result_path = project_root+"/represent/result/snowdamage/RESULT_MAP_e_0.814_in_1_s_False_l_[3]_e_[5, 6, 7, 8]_o_41_m_11_s_True_2.0.png"

disturbance_label_image = read_image(ground_truth_disturbance_path, is_satellite=False,is_cut=False)[:,:,0].astype(int)
intact_label_image = read_image(ground_truth_intact_path, is_satellite=False,is_cut=False)[:,:,0].astype(int)
forest_mask_image = read_image(forest_mask_path, is_satellite=False,is_cut=False)[:,:,0].astype(int)

result_image = plt.imread(result_path)

## Investigation on Data and Label

### Visualizing Pre-Change, Post-Change, and Ground Truth Images

In this section, we will visualize the pre-change image, post-change image, and the ground truth image with their respective cloud masks, gridlines, and scale bars. Additionally, we will create custom colorbars to represent forest thinning levels.

1. Plot the pre-change image and save it as `snowdamage_1a.jpg`.
2. Plot the post-change image and save it as `snowdamage_1b.jpg`.
3. Read the label image and filter out a cloudy or non-analysis area.
4. Create the plot with the label image and the non-analysis area mask, and save it as `snowdamage_1c.jpg`.


In [ ]:
# Plot the pre-change image
pre_input=plot_rgb_image(pre_s1_path)
plt.savefig("snowdamage_1a.jpg", dpi=144)

# Plot the post-change image
post_input=plot_rgb_image(post_s1_path)
plt.savefig("snowdamage_1b.jpg", dpi=144)

In [ ]:


# Define a colormap for the labels
cmap = ListedColormap(['black', 'green', 'red'])
bounds = [-0.5, 0.5, 1.5, 2.5]
norm = plt.Normalize(vmin=-0.5, vmax=2.5)

# Create a gray-scale image for the forest mask
forest_mask_gray = np.zeros((forest_mask_image.shape[0], forest_mask_image.shape[1]), dtype=np.uint8)
forest_mask_gray[forest_mask_image == 1] = 255 # white for forest
forest_mask_gray[forest_mask_image == 0] = 211 # black for non-forest

# Create an RGB image with the forest mask in the background
rgb_image = np.zeros((forest_mask_image.shape[0], forest_mask_image.shape[1], 3), dtype=np.uint8)
rgb_image[:,:,0] = forest_mask_gray
rgb_image[:,:,1] = forest_mask_gray
rgb_image[:,:,2] = forest_mask_gray

# Set label colors separately
rgb_image[intact_label_image == 1] = [0, 128, 0] # green for intact
rgb_image[disturbance_label_image == 1] = [255, 0, 0] # red for disturbance

# Create a legend with colors inside the image
legend_elements = [Patch(facecolor='green',edgecolor='gray', label='Intact Area'),
                   Patch(facecolor='red',edgecolor='gray', label='Disturbance Area'),
                   Patch(facecolor=(0.8, 0.8, 0.8, 1),edgecolor='gray', label='No Analysis Area')]

# Read the forest mask
with rasterio.open(forest_mask_path) as src:
    forest_mask = src.read(1)
    extent = src.bounds
    pixel_size = src.res[0]
    #dx, dy = transform.a, -transform.e
    dx = (extent[2] - extent[0]) / (forest_mask.shape[1])

# Calculate the aspect ratio
height, width = forest_mask.shape
aspect_ratio = float(width) / float(height)

# Calculate the figure size based on the aspect ratio
base_fig_size = 14
fig_size = (base_fig_size * aspect_ratio, base_fig_size)

# Enlarge the extent of the plot by 100% on each dimension
x_min, y_min, x_max, y_max = extent
x_range = x_max - x_min
y_range = y_max - y_min
extent = (x_min - x_range, x_max + x_range, y_min - y_range, y_max + y_range)

# Create the plot with the forest mask image only
fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': ccrs.epsg(3067)})
ax.imshow(rgb_image, cmap='gray', extent=extent, transform=ccrs.epsg(3067), origin='upper')

# Set the extent of the plot to include the entire forest mask image
ax.set_extent(extent, crs=ccrs.epsg(3067))

# Add gridlines and lat/long labels
gridlines = ax.gridlines(draw_labels=True)
gridlines.xformatter = LongitudeFormatter()
gridlines.yformatter = LatitudeFormatter()

# Add legend
ax.legend(handles=legend_elements, loc='upper right')

# Define the scale bar properties
scalebar = ScaleBar(1, 'm', length_fraction=0.25, height_fraction=0.005, location='lower right')

# Add the scale bar to the plot
ax.add_artist(scalebar)

 
#plt.show()
plt.tight_layout()
plt.savefig("snowdamage_1c.jpg", dpi=144)


## Snowdamage Segmentation Metrics

### Confusion Matrix Visualization

In this section, we will visualize the confusion matrix using an RGB image. We will calculate the binary masks for true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) from the binary label and predicted images. We will create an empty RGB image and assign the colors to each pixel based on the binary masks. Finally, we will plot the RGB image with a legend for the different colors.



In [ ]:



# Create a gray-scale image for the forest mask
forest_mask_gray = np.zeros((forest_mask_image.shape[0], forest_mask_image.shape[1]), dtype=np.uint8)
forest_mask_gray[forest_mask_image == 1] = 255 # white for forest
forest_mask_gray[forest_mask_image == 0] = 211 # black for non-forest

# Create an RGB image with the forest mask in the background
rgb_image = np.zeros((forest_mask_image.shape[0], forest_mask_image.shape[1], 3), dtype=np.uint8)
rgb_image[:,:,0] = forest_mask_gray
rgb_image[:,:,1] = forest_mask_gray
rgb_image[:,:,2] = forest_mask_gray

# Get the dimensions of the result image
height, width, _ = result_image.shape

# Get the first channel of the result image (predicted disturbance)
disturbance_array = result_image[:, :, 0]

# Get the second channel of the result image (predicted intact)
intact_array = result_image[:, :, 1]

# Create a mask of correctly classified disturbance pixels
correct_disturbance_mask = np.zeros((height, width), dtype=np.uint8)
correct_disturbance_mask[(disturbance_array >= 0.5) & (disturbance_label_image == 1)] = 1 # magenta for correctly classified disturbance pixels

# Create a mask of correctly classified intact pixels
correct_intact_mask = np.zeros((height, width), dtype=np.uint8)
correct_intact_mask[(intact_array >= 0.5) & (intact_label_image == 1)] = 2 # cyan for correctly classified intact pixels

# Create a mask of incorrectly classified disturbance pixels
incorrect_disturbance_mask = np.zeros((height, width), dtype=np.uint8)
incorrect_disturbance_mask[(disturbance_array < 0.5) & (disturbance_label_image == 1)] = 3 # orange for incorrectly classified disturbance pixels

# Create a mask of incorrectly classified intact pixels
incorrect_intact_mask = np.zeros((height, width), dtype=np.uint8)
incorrect_intact_mask[(intact_array < 0.5) & (intact_label_image == 1)] = 4 # yellow for incorrectly classified intact pixels

# Combine the masks into a single mask
mask = correct_disturbance_mask + correct_intact_mask + incorrect_disturbance_mask + incorrect_intact_mask

# Set the mask colors
rgb_image[mask == 1] = [0, 128, 0] # green for correctly classified disturbance
rgb_image[mask == 2] = [0, 255, 255] # cyan for correctly classified intact
rgb_image[mask == 3] = [255, 165, 0] # orange for incorrectly classified disturbance
rgb_image[mask == 4] = [255, 0, 255] # magenta for incorrectly classified intact

# Update the legend elements
legend_elements = [Patch( facecolor='green',edgecolor='gray', label='Correctly-detected Disturbance (TP)'),
                   Patch( facecolor='cyan',edgecolor='gray', label='Correctly-detected Intact (TN)'),
                   Patch( facecolor='orange',edgecolor='gray', label='Missed Disturbance (FN)'),
                   Patch( facecolor='magenta',edgecolor='gray', label='Missed Intact (FP)'),
                   #plt.Rectangle((0,0),1,1, color='black', label='Non-forest Mask'),
                  ]

# Read the forest mask
with rasterio.open(forest_mask_path) as src:
    forest_mask = src.read(1)
    extent = src.bounds

# Calculate the aspect ratio
height, width = forest_mask.shape
aspect_ratio = float(width) / float(height)

# Calculate the figure size based on the aspect ratio
base_fig_size = 14
fig_size = (base_fig_size * aspect_ratio, base_fig_size)

# Enlarge the extent of the plot by 100% on each dimension
x_min, y_min, x_max, y_max = extent
x_range = x_max - x_min
y_range = y_max - y_min
extent = (x_min - x_range, x_max + x_range, y_min - y_range, y_max + y_range)

# Create the plot with the forest mask image only
fig, ax = plt.subplots(figsize=fig_size, subplot_kw={'projection': ccrs.epsg(3067)})
ax.imshow(rgb_image, cmap='gray', extent=extent, transform=ccrs.epsg(3067), origin='upper')

ax.set_extent(extent, crs=ccrs.epsg(3067))

# Add gridlines and lat/long labels
gridlines = ax.gridlines(draw_labels=True)
gridlines.xformatter = LongitudeFormatter()
gridlines.yformatter = LatitudeFormatter()

# Add scalebar
scalebar = ScaleBar(1, 'm', length_fraction=0.25, height_fraction=0.005, location='lower right')
ax.add_artist(scalebar)


ax.legend(handles=legend_elements, loc='upper right')

#plt.show()
plt.tight_layout()
plt.savefig("snowdamage_1d.jpg", dpi=144)


### Zoomed Images and Result Plots

In this section, we will plot zoomed-in versions of the pre-change and post-change satellite images, as well as the resulting binary image and label image, focused on a specific area of interest.

We will set the bounding box for the area of interest using the north, west, east, and south coordinates. Then, we will use the plot_zoomed_input() function to plot the pre-change and post-change images, and plot_zoomed_result() functions to plot the resulting binary image and label image, respectively.

In [ ]:
north = 63.85
west = 29.5
east = 30
south = 63.6

plot_zoomed_input(pre_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_2a.jpg", dpi=144,bbox_inches='tight')
plot_zoomed_input(post_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_2b.jpg", dpi=144,bbox_inches='tight')


plot_zoomed_result(north, west, east, south,loc='upper right')
plt.savefig("snowdamage_2c.jpg", dpi=144,bbox_inches='tight')


In [ ]:
north = 65.6
west = 28
east = 28.5
south = 65.35

plot_zoomed_input(pre_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_3a.jpg", dpi=144,bbox_inches='tight')
plot_zoomed_input(post_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_3b.jpg", dpi=144,bbox_inches='tight')


plot_zoomed_result(north, west, east, south,loc='upper right')
plt.savefig("snowdamage_3c.jpg", dpi=144,bbox_inches='tight')


In [ ]:
north = 65.25
west = 29.
east = 29.5
south = 65

plot_zoomed_input(pre_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_4a.jpg", dpi=144,bbox_inches='tight')
plot_zoomed_input(post_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_4b.jpg", dpi=144,bbox_inches='tight')


plot_zoomed_result(north, west, east, south,loc='upper right')
plt.savefig("snowdamage_4c.jpg", dpi=144,bbox_inches='tight')


In [ ]:
north = 65.1
west = 29.5
east = 30.0
south = 64.85

plot_zoomed_input(pre_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_5a.jpg", dpi=144,bbox_inches='tight')
plot_zoomed_input(post_input,north, west, east, south,loc='upper right')
plt.savefig("snowdamage_5b.jpg", dpi=144,bbox_inches='tight')


plot_zoomed_result(north, west, east, south,loc='upper right')
plt.savefig("snowdamage_5c.jpg", dpi=144,bbox_inches='tight')


## ROC Analysis for Snowdamage on Sentinels

In this section, we read in four result tables for different layers of the model trained on Sentinel data to detect clearcuts. The tables are stored in JSON format and are loaded using pd.read_json(). We then call the process_and_plot() function to process the data and plot the ROC curves for each layer.

### Results based on BigEarthNet Model Weights 

In [ ]:
result_table_11=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.743_in_1_s_False_l_[1]_e_[5, 6, 7, 8]_o_27_m_11_s_True_2.0.json'))
result_table_12=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.713_in_1_s_False_l_[2]_e_[5, 6, 7, 8]_o_3_m_11_s_True_1.0.json'))
result_table_13=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.814_in_1_s_False_l_[3]_e_[5, 6, 7, 8]_o_41_m_11_s_True_2.0.json'))
result_table_14=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.668_in_1_s_False_l_[4]_e_[5, 6, 7, 8]_o_41_m_19_s_True_2.0.json'))

result_tables = [result_table_11, result_table_12, result_table_13, result_table_14]
labels = ['Layer 1', 'Layer 2', 'Layer 3', 'Layer 4']
title = 'ROC for Snow damage on Sentinel-1'
output_file = 'snowdamage_6a.jpg'
process_and_plot(result_tables, title, labels, output_file)

In [ ]:
result_table_11=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.788_in_1_s_False_l_[1]_e_[5, 6, 7, 8]_o_7_m_21_s_True_1.0.json'))
result_table_12=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.794_in_1_s_False_l_[2]_e_[5, 6, 7, 8]_o_21_m_23_s_True_2.0.json'))
result_table_13=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.813_in_1_s_False_l_[3]_e_[5, 6, 7, 8]_o_5_m_11_s_True_2.0.json'))
result_table_14=interpolate_dataframe(pd.read_json('/local_home/kuzu_ri/GIT_REPO/representlib/represent/result/snowdamage/resultComposite_e_0.801_in_1_s_False_l_[4]_e_[5, 6, 7, 8]_o_19_m_21_s_True_1.0.json'))

result_tables = [result_table_11, result_table_12, result_table_13, result_table_14]
labels = ['Layer 1', 'Layer 2', 'Layer 3', 'Layer 4']
title = 'ROC for Snow Damage on Sentinel-1'
output_file = 'snowdamage_6a.jpg'
process_and_plot(result_tables, title, labels, output_file)

In [ ]:
result_table_14=interpolate_dataframe(pd.read_json(project_root+'/represent/result/snowdamage/resultComposite_e_0.801_in_1_s_False_l_[4]_e_[5, 6, 7, 8]_o_19_m_21_s_True_1.0.json'))
result_table_24=interpolate_dataframe(pd.read_json(project_root+'/represent/result/snowdamage/resultComposite_e_0.763_in_1_s_False_l_[4]_e_[5, 6, 7, 8]_o_19_m_21_s_True_1.0.json'))
result_table_34=interpolate_dataframe(pd.read_json(project_root+'/represent/result/snowdamage/resultComposite_e_0.839_in_1_s_False_l_[4]_e_[5, 6, 7, 8]_o_19_m_21_s_True_1.0.json'))
result_table_44=(pd.read_json(project_root+'/represent/result/snowdamage/resultComposite_e_0.833_in_1_s_False_l_[4]_e_[5, 6, 7, 8]_o_19_m_21_s_True_1.0.json'))
result_table_54=(pd.read_json(project_root+'/represent/result/snowdamage/resultComposite_e_0.839_in_1_s_False_l_[4]_e_[5, 6, 7, 8]_o_19_m_21_s_True_11.0.json'))


result_tables = [result_table_14, result_table_24,result_table_34,result_table_44,result_table_54]
labels = ['BigEarthNet', 'PixPro','PixContrast', 'SimSiam', 'BYOL']
title = 'ROC for Snow Damage on Sentinel-1'
output_file = 'snowdamage_6b.jpg'
process_and_plot(result_tables, title, labels, output_file)

In [ ]:
result_table_34.sort_values(by=['idx'],ascending=False)

In [ ]:
np.trapz(result_table_34.to_numpy()[:, 1], x=1-result_table_34.to_numpy()[:, 2])